# **Instalación e Importación de Librerías**

In [1]:
!pip install seaborn  --upgrade
!pip install surprise --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357282 sha256=12cafb06b32400d1fc1a3f2a206246820130c57e9de9f82f9e79ae477f62c4d1
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import pandas as pd
import surprise
import seaborn as sns
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import time
import psutil
from tabulate import tabulate
from collections import defaultdict
import random


# **Carga de Datos**

In [3]:
!wget -O LLM-ReDial-2024.zip "https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1"
!unzip LLM-ReDial-2024.zip
!unzip LLM_Redial.zip
!rm LLM_Redial.zip
!rm LLM-ReDial-2024.zip
!rm -rf __MACOSX

--2024-10-14 21:29:53--  https://www.dropbox.com/scl/fi/x9avfdx2a1k6uq97f0efj/LLM-ReDial-2024.zip?rlkey=ijqpf91d13d6lowek3ebjvd0n&e=2&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5c67f348d83c687642c2d147ea.dl.dropboxusercontent.com/cd/0/inline/Ccd8ppFOnr8Xc1EzpoX1pY7wq4H4k8ipMnj6CYjDMhhU6aD4b1vCWCWGE9jIXskMYBWFfUoPZE1ieKaP_Y--K28qz3v1wRTCuusGxN5W2iGNP42nm2oG5AZ-7-rnrYxhHd8/file?dl=1# [following]
--2024-10-14 21:29:54--  https://uc5c67f348d83c687642c2d147ea.dl.dropboxusercontent.com/cd/0/inline/Ccd8ppFOnr8Xc1EzpoX1pY7wq4H4k8ipMnj6CYjDMhhU6aD4b1vCWCWGE9jIXskMYBWFfUoPZE1ieKaP_Y--K28qz3v1wRTCuusGxN5W2iGNP42nm2oG5AZ-7-rnrYxhHd8/file?dl=1
Resolving uc5c67f348d83c687642c2d147ea.dl.dropboxusercontent.com (uc5c67f348d83c687642c2d147ea.dl.dropboxusercontent.com)... 162.125.3.15, 2620:100:6018:15::a27

# **Random Baseline**

**Función para leer datos de un usuario**

In [4]:
### Directamente desde sample_data/Tools.py
import json
import csv

def read_user_data(filename, user_id):
    with open(filename, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]

    for entry in data:
        if user_id in entry:
            return entry[user_id]

    return None


**Función para recomendar aleatoriamente un item nuevo a un usuario**

In [5]:
### Retorna una lista con n recomendaciones, 0 si el usuario no se encuentra
def random_recommendations(userID, itemsID, final_data_path, n):
    user_data = read_user_data(final_data_path, userID)

    if user_data is None:
        print(f"No se encontraron datos para el usuario {userID}.")
        return []
    else:
        user_history = user_data['history_interaction']
        possible_recommendations = set(itemsID) - set(user_history)
        recommended_items = list(np.random.choice(list(possible_recommendations), size=n, replace=False))
        return recommended_items

**Evaluar baseline**

In [12]:
def evaluate_random_baseline(items_map_path, users_map_path, final_data_path, n, sample_percent=0.1):
    users_map = json.load(open(users_map_path, 'r'))
    items_map = json.load(open(items_map_path, 'r'))

    user_ids = list(users_map.keys())
    items_ids = list(items_map.keys())

    sample_size = int(sample_percent * len(user_ids))
    sampled_users = random.sample(user_ids, sample_size)

    hit = 0
    total = 0

    for user in sampled_users:
        user_data = read_user_data(final_data_path, user)
        if not user_data:
            continue

        user_conversations = user_data.get('Conversation', [])
        for j, conversation in enumerate(user_conversations):
            rec_item = conversation.get(f'conversation_{j+1}', {}).get('rec_item')
            if not rec_item:
                continue

            random_item = random_recommendations(user, items_ids, final_data_path, n)

            if rec_item[0] in random_item:
                hit += 1
            total += 1

    return hit / total if total > 0 else 0

In [13]:
movies_items_map_path = "Movie/item_map.json"
movies_users_map_path = "Movie/user_ids.json"
movies_final_data_path = "Movie/final_data.jsonl"

books_items_map_path = "Books/item_map.json"
books_users_map_path = "Books/user_ids.json"
books_final_data_path = "Books/final_data.jsonl"

**Evaluar para películas**

In [14]:
evaluate_random_baseline(movies_items_map_path, movies_users_map_path, movies_final_data_path, 5)

0.0

**Evaluar para libros**

In [15]:
evaluate_random_baseline(books_items_map_path, books_users_map_path, books_final_data_path, 5)

0.0